In [20]:
import json
import ast
import copy

## part 1 ##

In [18]:
teststr = '[[[9,[3,8]],[[0,9],6]],[[[3,7],[4,9]],3]]'

In [4]:
json.loads('[[[9,[3,8]],[[0,9],6]],[[[3,7],[4,9]],3]]')

[[[9, [3, 8]], [[0, 9], 6]], [[[3, 7], [4, 9]], 3]]

In [7]:
def addpairs(pair1, pair2):
    return [pair1, pair2]

In [8]:
addpairs([1,2],[3,4])

[[1, 2], [3, 4]]

In [9]:
pair = json.loads('[[[9,[3,8]],[[0,9],6]],[[[3,7],[4,9]],3]]')

In [10]:
lhs, rhs = pair
type(lhs), type(rhs)

(list, list)

In [11]:
lhs

[[9, [3, 8]], [[0, 9], 6]]

In [12]:
rhs

[[[3, 7], [4, 9]], 3]

In [13]:
import ast

In [15]:
ast.literal_eval('[[[9,[3,8]],[[0,9],6]],[[[3,7],[4,9]],3]]')

[[[9, [3, 8]], [[0, 9], 6]], [[[3, 7], [4, 9]], 3]]

In [17]:
class SfNum:
    def __init__(self, nums):
        self.nums = copy.deepcopy(nums)

    @classmethod
    def from_text(cls, text):
        return cls(ast.literal_eval(text))
    
    def __str__(self):
        return str(self.nums)

    def copy(self):
        return SfNum(self.nums)

    def walk(self):
        """Walk from left to right, yielding idx, kind, value."""
        yield from self._walk((), self.nums)

    def _walk(self, idx, nums):
        if isinstance(nums, int):
            yield "num", idx, nums
        else:
            yield "pair", idx, nums
            yield from self._walk(idx + (0,), nums[0])
            yield from self._walk(idx + (1,), nums[1])

    def __getitem__(self, idx):
        val = self.nums
        for i in idx:
            val = val[i]
        return val

    def __setitem__(self, idx, val):
        nums = self.nums
        for i in idx[:-1]:
            nums = nums[i]
        nums[idx[-1]] = val

    def reduce(self):
        while True:
            # Find a pair to explode:
            happened = False
            left_num = left_num_idx = add_to_right = explode_idx = None
            walking = self.walk()
            for kind, idx, val in walking:
                if kind == "num":
                    left_num = val
                    left_num_idx = idx
                elif len(idx) == 4:
                    # This is the pair to explode 
                    if left_num_idx is not None:
                        self[left_num_idx] = left_num + val[0]
                    add_to_right = val[1]
                    explode_idx = idx
                    happened = True
                    break
            if happened:
                # skip the two numbers in the exploded pair
                next(walking); next(walking)
                for kind, idx, val in walking:
                    if kind == "num":
                        self[idx] = val + add_to_right
                        break
            if happened:
                self[explode_idx] = 0
                continue

            # Find a number to split:
            happened = False
            for kind, idx, val in self.walk():
                if kind == "num" and val >= 10:
                    self[idx] = [math.floor(val / 2), math.ceil(val / 2)]
                    happened = True
                    break
            if not happened:
                break

    def __add__(self, other):
        added = SfNum([self.nums, other.nums])
        added.reduce()
        return added

    def magnitude(self):
        return magnitude(self.nums)

In [21]:
testpair = SfNum.from_text(teststr)

In [22]:
testpair

In [23]:
testpair.nums

[[[9, [3, 8]], [[0, 9], 6]], [[[3, 7], [4, 9]], 3]]

In [24]:
print(testpair)

[[[9, [3, 8]], [[0, 9], 6]], [[[3, 7], [4, 9]], 3]]


In [29]:
it = testpair.walk()
for pair in it:
    print(pair)

('pair', (), [[[9, [3, 8]], [[0, 9], 6]], [[[3, 7], [4, 9]], 3]])
('pair', (0,), [[9, [3, 8]], [[0, 9], 6]])
('pair', (0, 0), [9, [3, 8]])
('num', (0, 0, 0), 9)
('pair', (0, 0, 1), [3, 8])
('num', (0, 0, 1, 0), 3)
('num', (0, 0, 1, 1), 8)
('pair', (0, 1), [[0, 9], 6])
('pair', (0, 1, 0), [0, 9])
('num', (0, 1, 0, 0), 0)
('num', (0, 1, 0, 1), 9)
('num', (0, 1, 1), 6)
('pair', (1,), [[[3, 7], [4, 9]], 3])
('pair', (1, 0), [[3, 7], [4, 9]])
('pair', (1, 0, 0), [3, 7])
('num', (1, 0, 0, 0), 3)
('num', (1, 0, 0, 1), 7)
('pair', (1, 0, 1), [4, 9])
('num', (1, 0, 1, 0), 4)
('num', (1, 0, 1, 1), 9)
('num', (1, 1), 3)


In [26]:
next(it)

('pair', (), [[[9, [3, 8]], [[0, 9], 6]], [[[3, 7], [4, 9]], 3]])

In [27]:
next(it)

('pair', (0,), [[9, [3, 8]], [[0, 9], 6]])

In [28]:
next(it)

('pair', (0, 0), [9, [3, 8]])